In [1]:
CSV_PATH = "C:/Users/breastcancer.csv"
APP_NAME = "BDM2"
SPARK_URL = "local[*]"
RANDOM_SEED = 13579
TRAINING_DATA_RATIO = 0.7
MAX_DEPTH = 4
NUM_BINS = 32

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName(APP_NAME) \
    .master(SPARK_URL) \
    .getOrCreate()

#Acquisition des données sous forme de dataframe
df = spark.read \
    .options(header = "true", inferschema = "true") \
    .csv(CSV_PATH)
    

print("Nombre total de lignes: %d" % df.count())

Nombre total de lignes: 699


In [3]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint


#Prétraitement et transformation des données
#Suppression des données manquantes qui se trouvent au niveau de la 6eme colonne
transformed_df = df.rdd.filter(lambda x: x[6] != "?").map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
splits = [TRAINING_DATA_RATIO, 1.0 - TRAINING_DATA_RATIO]

#Diviser le dataset en dataset de test et dataset de traitement
training_data, test_data = transformed_df.randomSplit(splits, RANDOM_SEED)

print("Nombre de lignes dans le dataset d'entrainement: %d" % training_data.count())
print("Nombre de lignes dans le dataset de test: %d" % test_data.count())

Nombre de lignes dans le dataset d'entrainement: 447
Nombre de lignes dans le dataset de test: 236


In [4]:
from pyspark.mllib.tree import DecisionTree
from time import *

start_time = time()

#Traitement : Construction du modèle Arbre de Décision
model = DecisionTree.trainClassifier(training_data, numClasses=5, categoricalFeaturesInfo={}, \
    impurity="gini", maxDepth=MAX_DEPTH, maxBins=NUM_BINS)

end_time = time()
elapsed_time = end_time - start_time
print("Temps d'entrainement du dataset: %.3f seconds" % elapsed_time)

Temps d'entrainement du dataset: 4.858 seconds


In [5]:
#Evaluation du modèle

predictions = model.predict(test_data.map(lambda x: x.features))
labels_and_predictions = test_data.map(lambda x: x.label).zip(predictions)
acc = labels_and_predictions.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())
print("Précision du modèle: %.3f%%" % (acc * 100))

Précision du modèle: 93.220%


In [6]:
print (model.toDebugString())

DecisionTreeModel classifier of depth 4 with 17 nodes
  If (feature 2 <= 2.5)
   If (feature 8 <= 3.5)
    Predict: 2.0
   Else (feature 8 > 3.5)
    If (feature 1 <= 3.5)
     Predict: 2.0
    Else (feature 1 > 3.5)
     Predict: 4.0
  Else (feature 2 > 2.5)
   If (feature 6 <= 1.5)
    If (feature 4 <= 3.5)
     Predict: 2.0
    Else (feature 4 > 3.5)
     Predict: 4.0
   Else (feature 6 > 1.5)
    If (feature 3 <= 2.5)
     If (feature 0 <= 1133563.5)
      Predict: 4.0
     Else (feature 0 > 1133563.5)
      Predict: 2.0
    Else (feature 3 > 2.5)
     If (feature 6 <= 8.5)
      Predict: 4.0
     Else (feature 6 > 8.5)
      Predict: 4.0

